앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.

이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").

항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.

메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.


In [1]:
# 필요한 라이브러리 import
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory

print("✅ 라이브러리 import 완료!")

✅ 라이브러리 import 완료!


In [2]:
# 영화 이모티콘 예시 데이터 준비
examples = [
    {"input": "탑건", "output": "🛩️👨‍✈️🔥"},
    {"input": "대부", "output": "👨‍👨‍👦🔫🍝"},
    {"input": "타이타닉", "output": "🚢💔🌊"},
    {"input": "어벤져스", "output": "🦸‍♂️💥⚡"},
]

# FewShotChatMessagePromptTemplate 구성
example_prompt = ChatPromptTemplate.from_messages([("human", "{input}"), ("ai", "{output}")])

few_shot_prompt = FewShotChatMessagePromptTemplate(example_prompt=example_prompt, examples=examples)

# MessagesPlaceholder를 사용한 최종 프롬프트 템플릿
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 영화를 세 개의 이모티콘으로 표현하는 전문가입니다. 주어진 영화 제목에 대해 영화의 핵심 요소를 나타내는 세 개의 이모티콘만 답변하세요.",
        ),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

print("✅ FewShotChatMessagePromptTemplate과 MessagesPlaceholder 구성 완료!")

✅ FewShotChatMessagePromptTemplate과 MessagesPlaceholder 구성 완료!


In [3]:
# ChatMessageHistory를 사용한 메모리 설정
chat_history = ChatMessageHistory()
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
output_parser = StrOutputParser()


# ChatMessageHistory를 활용한 메모리 체인 구성
def load_memory(inputs):
    """ChatMessageHistory에서 이전 대화를 로드합니다."""
    return {"input": inputs["input"], "chat_history": chat_history.messages}


def save_memory(inputs, outputs):
    """대화를 ChatMessageHistory에 저장합니다."""
    chat_history.add_user_message(inputs["input"])
    chat_history.add_ai_message(outputs)
    return outputs


# RunnablePassthrough를 사용한 LCEL 체인 (올바른 방식)
movie_emoji_chain = (
    RunnablePassthrough.assign(chat_history=lambda x: chat_history.messages)
    | final_prompt
    | model
    | output_parser
)

print("✅ ChatMessageHistory를 활용한 메모리 체인 구성 완료!")

✅ ChatMessageHistory를 활용한 메모리 체인 구성 완료!


In [4]:
# 첫 번째 영화 테스트
print("🎬 첫 번째 영화 테스트")
print("=" * 30)

movie1 = "인터스텔라"
print(f"영화: {movie1}")

result1 = movie_emoji_chain.invoke({"input": movie1})
print(f"이모티콘: {result1}")

# 메모리에 저장
save_memory({"input": movie1}, result1)
print("✅ 메모리에 저장됨")

🎬 첫 번째 영화 테스트
영화: 인터스텔라
이모티콘: 🚀⏳🌌
✅ 메모리에 저장됨


In [5]:
# 두 번째 영화 테스트
print("🎬 두 번째 영화 테스트")
print("=" * 30)

movie2 = "겨울왕국"
print(f"영화: {movie2}")

result2 = movie_emoji_chain.invoke({"input": movie2})
print(f"이모티콘: {result2}")

# 메모리에 저장
save_memory({"input": movie2}, result2)
print("✅ 메모리에 저장됨")

🎬 두 번째 영화 테스트
영화: 겨울왕국
이모티콘: ❄️👸⛄
✅ 메모리에 저장됨


In [6]:
# ChatMessageHistory 테스트 - 이전에 질문한 영화 확인
print("🧠 ChatMessageHistory 테스트")
print("=" * 30)

# ChatMessageHistory 내용 확인
print("📝 ChatMessageHistory에 저장된 대화:")
for i, message in enumerate(chat_history.messages, 1):
    if hasattr(message, 'content'):
        print(f"{i}. {message.content}")
    else:
        print(f"{i}. {message}")

print(f"\n💭 총 {len(chat_history.messages)}개의 메시지가 ChatMessageHistory에 저장되어 있습니다.")

🧠 ChatMessageHistory 테스트
📝 ChatMessageHistory에 저장된 대화:
1. 인터스텔라
2. 🚀⏳🌌
3. 겨울왕국
4. ❄️👸⛄

💭 총 4개의 메시지가 ChatMessageHistory에 저장되어 있습니다.


In [7]:
# ChatMessageHistory 기능 테스트 - 이전 질문 기억하기
print("🔍 ChatMessageHistory 기능 테스트")
print("=" * 30)

# 이전에 질문한 영화가 무엇인지 물어보기
question = "이전에 질문한 영화가 무엇인지 알려주세요"
print(f"질문: {question}")

# ChatMessageHistory를 사용한 체인으로 답변
memory_test_result = movie_emoji_chain.invoke({"input": question})
print(f"답변: {memory_test_result}")

# 이 질문도 ChatMessageHistory에 저장
save_memory({"input": question}, memory_test_result)

print("\n✅ ChatMessageHistory가 정상적으로 작동하는지 확인되었습니다!")

🔍 ChatMessageHistory 기능 테스트
질문: 이전에 질문한 영화가 무엇인지 알려주세요
답변: 이전에 질문한 영화는 '인터스텔라'입니다.

✅ ChatMessageHistory가 정상적으로 작동하는지 확인되었습니다!
